데이터 전처리

In [2]:
import pandas as pd
import torch
from utils import *

# Load dataset
region="seoul"
path = f"../../collect_data/filtered/kma/merged/kma_{region}_meta.csv"

# 2. CSV 파일 읽어서 데이터프레임으로 변환
korea_data = pd.read_csv(path)

# Select relevant columns for prediction
#selected_columns = ['STN','TM', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'CA_TOT', 'CA_MID', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02','TE_03', 'PV', 'PM10']
#korea_data = korean_df[selected_columns]

# Sort by time
#korea_data = korea_data.sort_values('TM').reset_index(drop=True)

korea_data = convert_timesteps(korea_data)
print_missing_info(korea_data)

print(korea_data)

# 데이터셋 인스턴스를 생성합니다.
# window_size = 56
# prediction_length = 28
# korea_data = FinedustDataset(korea_data,window_size=window_size, prediction_length=prediction_length)

(40175, 26)
Series([], dtype: float64)
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV', 'WW',
       'CA_TOT', 'CA_MID', 'CT', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02',
       'TE_03', 'IR', 'PM10', '지점', '위도', '경도', 'diff'],
      dtype='object')
                       TM    STN    WD   WS      PA      PS   TA    TD    HM  \
0     2018-01-01 00:00:00  108.0   0.0  0.0     0.0     0.0  0.0   0.0   0.0   
1     2018-01-01 01:00:00  108.0   0.0  0.0     0.0     0.0  0.0   0.0   0.0   
2     2018-01-01 02:00:00  108.0   0.0  0.0     0.0     0.0  0.0   0.0   0.0   
3     2018-01-01 03:00:00  108.0   0.0  0.0     0.0     0.0  0.0   0.0   0.0   
4     2018-01-01 04:00:00  108.0   0.0  0.0     0.0     0.0  0.0   0.0   0.0   
...                   ...    ...   ...  ...     ...     ...  ...   ...   ...   
40170 2022-11-30 20:00:00  108.0  27.0  3.1  1022.6  1033.8 -5.8  13.4  55.0   
40171 2022-11-30 21:00:00  108.0  27.0  4.2  1022.6  1033.8 -6.7  12.9  61.0   
40172 2022-11-3

Transformer - 한국만

In [3]:
import tensorflow as tf

# Timeseries Transformer 인코더 층 정의
def timeseries_transformer_encoder(inputs, num_heads, key_dim, ff_dim, dropout_rate):
    attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, dropout=dropout_rate)(inputs, inputs)
    attention = tf.keras.layers.Dropout(dropout_rate)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)
    
    ff = tf.keras.layers.Dense(ff_dim, activation='relu')(attention)
    ff = tf.keras.layers.Dense(inputs.shape[-1])(ff)  # 입력과 동일한 크기로 설정
    ff = tf.keras.layers.Dropout(dropout_rate)(ff)
    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + ff)

In [4]:
from sklearn.preprocessing import StandardScaler

# 윈도우 크기, predicition_size 설정
window_size = 30  # 윈도우 크기
prediction_length = 7  # 예측하려는 시간 범위
scaler = StandardScaler()

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

removed_columns = ['TM', 'PM10', 'WW', 'CT']

# 데이터 준비
X = korea_data.drop(columns=removed_columns)
X.fillna(X.mean(), inplace=True)  # 결측값 처리
y = korea_data['PM10']

# 데이터 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 데이터 정규화 및 시퀀스
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_test_scaled = scaler.transform(X_test)

X_train_val_seq = np.array([X_train_val_scaled[i:i+window_size] for i in range(len(X_train_val_scaled) - window_size - prediction_length + 1)])
y_train_val_seq = np.array([y_train_val.values[i+window_size:i+window_size+prediction_length] for i in range(len(X_train_val_seq))])

X_test_seq = np.array([X_test_scaled[i:i+window_size] for i in range(len(X_test_scaled) - window_size - prediction_length + 1)])
y_test_seq = np.array([y_test.values[i+window_size:i+window_size+prediction_length] for i in range(len(X_test_seq))])

# Transformer 모델 정의 및 구성
input_dim = X_train_val_seq.shape[2]  # Feature dimension

inputs = tf.keras.Input(shape=(window_size, X_train_val_seq.shape[2]))
x = timeseries_transformer_encoder(inputs, num_heads=4, key_dim=64, ff_dim=128, dropout_rate=0.1)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(prediction_length)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습 및 평가
history = model.fit(
    X_train_val_seq, y_train_val_seq,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=0
)

test_loss, test_mae = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Set Mean Absolute Error: {test_mae}")

# 예측 (test_size, prediction_length)
predictions = model.predict(X_test_seq)


250/250 [==============================] - 1s 3ms/step - loss: 605.3649 - mae: 17.2852
Test Set Mean Absolute Error: 17.28523826599121
250/250 [==============================] - 1s 3ms/step


In [ ]:
import pandas as pd

# 예측,실제 값을 데이터프레임으로 변환
pred_df = pd.DataFrame(predictions, columns=[f"pred_{i+1}" for i in range(prediction_length)])
actual_df = pd.DataFrame(y_test_seq, columns=[f"actual_{i+1}" for i in range(prediction_length)])
comparison_df = pd.concat([pred_df, actual_df], axis=1)

# 일부 데이터 출력 (예: 상위 10개 행)
print(comparison_df.head(10))

# CSV 파일로 저장
# comparison_df.to_csv('predictions.csv', index=False)


      pred_1     pred_2     pred_3     pred_4     pred_5     pred_6  \
0  30.095184  30.886255  30.959536  31.761076  30.966103  31.675886   
1  29.996330  30.786329  30.859476  31.662292  30.875843  31.582747   
2  30.293821  31.116236  31.188841  31.960320  31.172642  31.895237   
3  30.710398  31.585386  31.650015  32.383030  31.591175  32.329094   
4  31.113224  32.041492  32.098732  32.793652  31.996870  32.751186   
5  31.533499  32.513126  32.562763  33.220257  32.416763  33.186382   
6  32.187664  33.211559  33.244652  33.865009  33.031746  33.815987   
7  32.689610  33.673145  33.703785  34.326862  33.451538  34.208961   
8  33.061996  34.060818  34.092365  34.692833  33.808407  34.551178   
9  33.161102  34.194794  34.230717  34.802116  33.933777  34.679897   

      pred_7  actual_1  actual_2  actual_3  actual_4  actual_5  actual_6  \
0  32.711861      44.0      41.0      48.0      59.0      63.0      66.0   
1  32.617149      41.0      48.0      59.0      63.0      66.0    

Transformer-중국포함

In [7]:
korea_data = pd.read_csv(path)
china_data = concat_china_data(korea_data)

china_data = convert_timesteps(china_data)
print_missing_info(china_data)

print(china_data)

(9488, 111)
VV             0.031619
WD_yanan       0.021079
WS_yanan       0.021079
TA_yanan       0.105396
TD_yanan       0.274030
HM_yanan       0.685076
PS_yanan       0.031619
PR             0.063238
VV_tongliao    0.010540
WD_tongliao    0.010540
WS_tongliao    0.010540
TA_tongliao    0.137015
TD_tongliao    0.252951
HM_tongliao    0.242411
PS_tongliao    0.010540
PR_tongliao    0.010540
VV_qingdao     0.094857
WD_qingdao     0.115936
WS_qingdao     0.115936
TA_qingdao     0.094857
TD_qingdao     0.200253
HM_qingdao     0.790472
PS_qingdao     0.094857
PR_qingdao     0.094857
VV_chifeng     0.063238
WD_chifeng     0.021079
WS_chifeng     0.063238
TA_chifeng     0.115936
TD_chifeng     0.263491
HM_chifeng     0.179174
PS_chifeng     0.031619
PR_chifeng     0.021079
TA_dalian      0.042159
TD_dalian      0.242411
HM_dalian      0.042159
dtype: float64
Index(['TM', 'STN', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'PV',
       ...
       'TD_dalian', 'HM_dalian', 'PS_dalian', 'PT_dali

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

removed_columns = ['TM', 'PM10', 'WW', 'CT']

# 데이터 준비
X = china_data.drop(columns=removed_columns)
X.fillna(X.mean(), inplace=True)  # 결측값 처리
y = china_data['PM10']

# 데이터 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 데이터 정규화 및 시퀀스
X_train_val_scaled = scaler.fit_transform(X_train_val)
X_test_scaled = scaler.transform(X_test)

X_train_val_seq = np.array([X_train_val_scaled[i:i+window_size] for i in range(len(X_train_val_scaled) - window_size - prediction_length + 1)])
y_train_val_seq = np.array([y_train_val.values[i+window_size:i+window_size+prediction_length] for i in range(len(X_train_val_seq))])

X_test_seq = np.array([X_test_scaled[i:i+window_size] for i in range(len(X_test_scaled) - window_size - prediction_length + 1)])
y_test_seq = np.array([y_test.values[i+window_size:i+window_size+prediction_length] for i in range(len(X_test_seq))])

# Transformer 모델 정의 및 구성
input_dim = X_train_val_seq.shape[2]  # Feature dimension

inputs = tf.keras.Input(shape=(window_size, X_train_val_seq.shape[2]))
x = timeseries_transformer_encoder(inputs, num_heads=4, key_dim=64, ff_dim=128, dropout_rate=0.1)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(prediction_length)(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습 및 평가
history = model.fit(
    X_train_val_seq, y_train_val_seq,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=0
)

test_loss, test_mae = model.evaluate(X_test_seq, y_test_seq)
print(f"Test Set Mean Absolute Error: {test_mae}")

# 예측 (test_size, prediction_length)
predictions = model.predict(X_test_seq)


59/59 [==============================] - 0s 4ms/step - loss: 803.3805 - mae: 21.3715
Test Set Mean Absolute Error: 21.371501922607422
59/59 [==============================] - 0s 4ms/step


In [11]:
import pandas as pd

# 예측,실제 값을 데이터프레임으로 변환
pred_df = pd.DataFrame(predictions, columns=[f"pred_{i+1}" for i in range(prediction_length)])
actual_df = pd.DataFrame(y_test_seq, columns=[f"actual_{i+1}" for i in range(prediction_length)])
comparison_df = pd.concat([pred_df, actual_df], axis=1)

# 일부 데이터 출력 (예: 상위 10개 행)
print(comparison_df.head(10))

# CSV 파일로 저장
comparison_df.to_csv('predictions.csv', index=False)


      pred_1     pred_2     pred_3     pred_4     pred_5     pred_6  \
0  14.306901  12.963779  12.793951  14.224319  14.885900  14.158140   
1  13.580867  12.232352  12.011200  13.515442  14.183243  13.576749   
2  11.594143  10.376992   9.816342  11.528031  12.357327  11.953491   
3  13.468301  12.247033  11.899059  13.344338  14.191192  13.270462   
4  13.439093  12.219550  11.862348  13.311944  14.168741  13.241842   
5  13.668106  12.439092  12.128189  13.551778  14.377426  13.447536   
6  13.468888  12.312323  11.965425  13.420453  14.238415  13.311279   
7  13.988493  12.801726  12.568789  13.963269  14.712805  13.758474   
8  14.531341  13.295014  13.187711  14.508088  15.192886  14.210098   
9  31.642080  28.832083  30.563902  30.711941  29.940981  28.497238   

      pred_7  actual_1  actual_2  actual_3  actual_4  actual_5  actual_6  \
0  13.431527       8.0      14.0      18.0       7.0      16.0      12.0   
1  12.729303      14.0      18.0       7.0      16.0      12.0    